# VARS-TOOL Quick Start Tutorial

## Install

This is a simple tutorial show casing VARS-TOOL functionalities that are being implement in Python.Let's install the latest VARS-TOOL from GitHub:

In [1]:
!git clone https://github.com/vars-tool/vars-tool
!pip install vars-tool/.

Cloning into 'vars-tool'...


Processing c:\users\corde\giws\vars-tool\tutorial\vars-tool
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for varstool: filename=varstool-2.1-py2.py3-none-any.whl size=427236 sha256=6f4e6dc428c9bc380c876a40f9db397c06c0cd1c2d8abc73db4580e1a62e3da2
  Stored in directory: c:\users\corde\appdata\local\pip\cache\wheels\ea\22\e6\94dfc4ddef885e03d97899daed3e63a6c2c3a1295f1b37f687
Successfully built varstool
  Attempting uninstall: varstool
    Found existing installation: varstool 2.1
    Uninstalling varstool-2.1:
      Successfully uninstalled varstool-2.1


## Example using the Ishigami and wavy6D models

Let's import VARS for the analysis and the Model class for creating a wrapper around the desired model so that it can be inputted into VARS

In [2]:
# importing VARS and Model from varstool package
from varstool import VARS, Model

# these are being imported in order for us to display the data nicely on the notebook
import numpy as np
import pandas as pd

Here is a quick function definition of the Ishigami model:

In [3]:
def ishigami(x, a=7, b=0.05):
    '''Ishigami test function'''
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 3:
        raise ValueError('`x` must have only three arguments at a time')
    
    return np.sin(x[0]) + a*(np.sin(x[1])**2) + b*(x[2]**4)*np.sin(x[0])

Here is a quick function definition of the wavy6D model:

In [4]:
def wavy6D(x):
    '''wavy6D test function'''
    
# The features include:             
# (1) The 6 input factors are non-interacting                  
# (2) The function is "wavy" along the first 5 dimensions but with different 
#     frequencies and amplitude at different directions                                     
# (3) The last dimension (X6) is fully isensitive (dummy variable)                                         
# (4) It possess large-scale features such multi-modality (X1) 
#     to small-scale features such as roughness (X3)          
# (5) The directional variograms of X1 and X2 cross each other
    
    # check whether the input x is a dataframe
    
    if not isinstance(x, (pd.core.frame.DataFrame, pd.core.series.Series, np.ndarray, list)):
        raise TypeError('`x` must be of type pandas.DataFrame, numpy.ndarray, pd.Series, or list')
    
    if len(x) > 6:
        raise ValueError('`x` must have only five arguments at a time')
    
    y1 = -1*np.sin(2*np.pi*x[0]/2) - 0.3*np.sin(2*np.pi*x[0]/0.6)
    y2 = 0.76*(-1*np.sin(2*np.pi*(x[1]-0.2)/2)) - 0.315
    y3 = 0.12*(-1*np.sin(2*np.pi*(x[2]-0.2)/1.9)) + 0.02*(-1*np.sin(2*np.pi*x[2]/0.021)) - 0.96
    y4 = 0.12*(-1*np.sin(2*np.pi*(x[3]-0.2)/1.9))-0.97
    y5 = 0.05*(-1*np.sin(2*np.pi*(x[4]-0.2)/2))-1.02
    y6 = -1.08

    return y1 + y2 + y3 + y4 + y5 + y6

As mentioned previously The `Model` class is a wrapper for custom functions(models) in the online version of VARS. However, the first paramter of every function must accept an array of parameters

Here we will create two wrappers for the testing models ishigami and wavy6D:

In [5]:
ishigami_model = Model(ishigami)

In [6]:
wavy6D_model = Model(wavy6D)

When creating an experiment you will need to assign a variable to an instance of VARS as shown below.

The paramters of the VARS class are described as:

**paramaters**: the name of each paramter along with their upper and lower bounds

**num_stars**: the total number of star samples that are desired for STAR-VARS analysis

**delta_h**: the sampling resolution of the VARS analysis

**ivars_scales**: the scales that are to be used when doing IVARS, e.g, 0.1 and 0.3 correspond (0-0.1) and (0-0.3) <br /> note: can not have a scale larger then 0.5

**star_centres**: This is only used if a sampler is not chosen and you are wanting to generate your own star centres(randomized numbers)

**sampler**: the sampling strategy: `rnd`, `lhs`, `plhs`, `sobol_seq`, or `halton_seq` for generation of star centers.

**seed**: the seed number for randomization of the sampling strategy specified by `sampler`, only needed if a sampler was chosen

**model**: the wrapper of your model you made when using the `Model` class

**bootstrap_flag**: this is a True/False value that specifies if bootstrapping will be used in the VARS analysis

**bootstrap_size**: the number of sampling iterations with replacement

**bootstrap_ci**: the bootstrap-based confidence interval on results

**grouping_flag**: this is a True/False value that specifies if paramater grouping will be used in the VARS analysis

**num_grps**: the number of groups you want to split your paramaters into, if left blank the optimal number of groups will be calculated by VARS

Create `experiment_1` and initialize the values needed to run a VARS analysis:

Run the next block if you want to run the notebook using the ishigami model, do not run it if you are using wavy6D

In [7]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = ishigami_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

Run the next block if you want to run the notebook using the wavy6D model do not run it if you are using ishigami

In [8]:
experiment_1 = VARS(parameters = {'x1':(0, 1), 'x2':(0, 1), 'x3': (0, 1), \
                                  'x4':(0, 1), 'x5':(0, 1), 'x6': (0, 1)},
                    num_stars=10,
                    delta_h = 0.1,
                    ivars_scales = (0.1, 0.3, 0.5),
                    sampler = 'rnd',
                    seed = 123456789,
                    model = wavy6D_model,
                    bootstrap_flag = True,
                    bootstrap_size = 100,
                    bootstrap_ci=0.9,
                    grouping_flag=True,
                    num_grps=2
                )

A report displaying the current status of the VARS analysis can be found by typing in the variable name of the instance you created, here this is `experiment_1`

In [9]:
experiment_1

Star Centres: Loaded
Star Points: Not Loaded
Parameters: 6 paremeters set
Delta h: 0.1
Model: wavy6D
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Not Done

To run the VARS analysis simply do the following:

In [10]:
experiment_1.run_online()

Now if we take a look at the status report again we can see that the `Vars Analysis` is now done.

In [11]:
experiment_1

Star Centres: Loaded
Star Points: Loaded
Parameters: 6 paremeters set
Delta h: 0.1
Model: wavy6D
Seed Number: 123456789
Bootstrap: On
Bootstrap Size: 100
Bootstrap CI: 0.9
Grouping: On
Number of Groups: 2
VARS Analysis: Done

Now we can access all the results using 'dot' notation on our created instance:

**Directional variogram results**

There are 10 rows as the number of stars was 10, and each at a resolution incremented by 0.1 which was specificed to be the sampling resolution

In [12]:
experiment_1.variogram_value.unstack(0)

param,x1,x2,x3,x4,x5,x6
h,,,,,,
0.1,0.045182,0.013609,0.000575,0.000384,0.000059,0.0
0.2,0.110273,0.050765,0.001864,0.001439,0.000217,0.0
0.3,0.120143,0.104139,0.003227,0.002960,0.000443,0.0
0.4,0.093506,0.165281,0.004640,0.004702,0.000697,0.0
0.5,0.097372,0.226212,0.006367,0.006426,0.000946,0.0
0.6,0.158481,0.280646,0.008212,0.007938,0.001164,0.0
0.7,0.220507,0.324558,0.009491,0.009108,0.001335,0.0
0.8,0.201650,0.356069,0.009745,0.009872,0.001454,0.0
0.9,0.113114,0.374792,0.009262,0.010212,0.001523,0.0


**Directional covariogram results**

In [13]:
experiment_1.covariogram_value.unstack(0)

param,x1,x2,x3,x4,x5,x6
h,,,,,,
0.1,0.192578,0.203769,0.137253,0.147068,0.140738,0.143339
0.2,0.133822,0.163402,0.135635,0.145346,0.140496,0.143339
0.3,0.125409,0.111817,0.134062,0.143820,0.140274,0.143339
0.4,0.153579,0.055488,0.132866,0.142737,0.140106,0.143339
0.5,0.163618,0.001775,0.132082,0.142331,0.140025,0.143339
0.6,0.123365,-0.041831,0.131653,0.142796,0.140059,0.143339
0.7,0.076505,-0.068499,0.131666,0.144263,0.140229,0.143339
0.8,0.126918,-0.072800,0.132508,0.146778,0.140547,0.143339
0.9,0.356940,-0.051326,0.134779,0.150292,0.141012,0.143339


**Directional expected covariogram results**

In [14]:
experiment_1.e_covariogram_value.unstack(0)

param,x1,x2,x3,x4,x5,x6
h,,,,,,
0.1,0.029078,0.101440,0.002878,0.002880,0.000421,5.522026e-31
0.2,-0.029614,0.057949,0.001373,0.001646,0.000237,5.522026e-31
0.3,-0.036576,0.006130,0.000012,0.000161,0.000019,5.522026e-31
0.4,-0.007359,-0.047305,-0.001171,-0.001371,-0.000203,5.522026e-31
0.5,0.002419,-0.094942,-0.002519,-0.002725,-0.000399,5.522026e-31
0.6,-0.038223,-0.129435,-0.003794,-0.003682,-0.000536,5.522026e-31
0.7,-0.082858,-0.144286,-0.004220,-0.004054,-0.000588,5.522026e-31
0.8,-0.025329,-0.134572,-0.003451,-0.003705,-0.000533,5.522026e-31
0.9,0.216465,-0.097533,-0.001854,-0.002574,-0.000361,5.522026e-31


**VARS-TO: Sobol total-order effects calculated through VARS**

In [15]:
experiment_1.sobol_value

param
x1    3.964245e-01
x2    6.141702e-01
x3    1.843206e-02
x4    1.742332e-02
x5    2.557823e-03
x6    2.947841e-30
dtype: float64

**VARS-ABE: Morris mean absolute elementary effects across scales**

In [16]:
experiment_1.morris_value[0].unstack(0)

param,x1,x2,x3,x4,x5,x6
h,,,,,,
0.1,0.234569,0.146612,0.028170,0.024748,0.009643,0.0
0.2,0.356113,0.279122,0.051531,0.047433,0.018244,0.0
0.3,0.357836,0.394207,0.068524,0.066685,0.025775,0.0
0.4,0.326848,0.491599,0.082793,0.083618,0.031826,0.0
0.5,0.367118,0.576076,0.096984,0.096991,0.037270,0.0
0.6,0.491983,0.660552,0.110845,0.111799,0.042040,0.0
0.7,0.595695,0.749198,0.125018,0.125671,0.047681,0.0
0.8,0.526273,0.816487,0.134129,0.136133,0.051964,0.0
0.9,0.438033,0.858505,0.133835,0.141886,0.054638,0.0


**VARS-ACE: Morris mean actual elementary effects across scales**

In [17]:
experiment_1.morris_value[1].unstack(0)

param,x1,x2,x3,x4,x5,x6
h,,,,,,
0.1,0.039893,-0.095389,-0.014871,-0.015765,-0.006071,0.0
0.2,0.069235,-0.204122,-0.033532,-0.034033,-0.012991,0.0
0.3,0.060101,-0.321085,-0.053579,-0.053859,-0.020435,0.0
0.4,0.019551,-0.440368,-0.072295,-0.074129,-0.028026,0.0
0.5,-0.007424,-0.555636,-0.090074,-0.093642,-0.035363,0.0
0.6,0.029326,-0.660552,-0.108443,-0.111194,-0.042040,0.0
0.7,0.140236,-0.749198,-0.125018,-0.125671,-0.047681,0.0
0.8,0.276941,-0.816487,-0.134129,-0.136133,-0.051964,0.0
0.9,0.359037,-0.858505,-0.133835,-0.141886,-0.054638,0.0


**Integrated variogram**

In [18]:
experiment_1.ivars

,x1,x2,x3,x4,x5,x6
0.1,0.002259,0.000680,0.000029,0.000019,0.000003,0.0
0.3,0.021553,0.011644,0.000405,0.000330,0.000050,0.0
0.5,0.041779,0.044690,0.001349,0.001270,0.000189,0.0


**The factor(parameter) rankings based on their influence**. 

The influence is based on how large or small a result is. The lower the ranking the more influential(larger) a factor is.

In [19]:
experiment_1.ivars_factor_ranking

,x1,x2,x3,x4,x5,x6
0.1,0,1,2,3,4,5
0.3,0,1,2,3,4,5
0.5,1,0,2,3,4,5


In [20]:
experiment_1.sobol_factor_ranking

,x1,x2,x3,x4,x5,x6
0,1,0,2,3,4,5


**Bootstrapping results based on the confidence interval limits**

low is the lower limit of the results after `bootstrap_size` sampling iterations

upp is the upper limit of the results after `bootstrap_size` sampling iterations

In [21]:
experiment_1.variogram_low

h,x1,x2,x3,x4,x5,x6
0.1,0.045147,0.013498,0.000526,0.000382,0.000058,0.0
0.2,0.105496,0.049842,0.001692,0.001417,0.000211,0.0
0.3,0.108478,0.101006,0.003056,0.002876,0.000421,0.0
0.4,0.082360,0.158078,0.004394,0.004500,0.000648,0.0
0.5,0.091282,0.213060,0.005792,0.006047,0.000856,0.0
0.6,0.154940,0.260178,0.007146,0.007341,0.001023,0.0
0.7,0.214013,0.296399,0.008053,0.008285,0.001141,0.0
0.8,0.183561,0.321105,0.008379,0.008857,0.001214,0.0
0.9,0.071746,0.335142,0.007688,0.009078,0.001251,0.0


In [22]:
experiment_1.variogram_upp

h,x1,x2,x3,x4,x5,x6
0.1,0.045226,0.013769,0.000617,0.000386,0.000059,0.0
0.2,0.115268,0.052103,0.002015,0.001463,0.000223,0.0
0.3,0.132589,0.108683,0.003386,0.003054,0.000462,0.0
0.4,0.106770,0.175728,0.004877,0.004928,0.000741,0.0
0.5,0.105545,0.245290,0.006853,0.006850,0.001027,0.0
0.6,0.163040,0.310334,0.009094,0.008605,0.001289,0.0
0.7,0.228643,0.365401,0.010869,0.010028,0.001507,0.0
0.8,0.227951,0.406781,0.011095,0.011006,0.001668,0.0
0.9,0.166295,0.432301,0.010744,0.011479,0.001766,0.0


In [23]:
experiment_1.sobol_low

param,x1,x2,x3,x4,x5,x6
,0.326486,0.52818,0.015287,0.014132,0.002141,2.072049e-30


In [24]:
experiment_1.sobol_upp

param,x1,x2,x3,x4,x5,x6
,0.5537,0.822403,0.025031,0.024404,0.00339,4.304406e-30


In [25]:
experiment_1.ivars_low

,x1,x2,x3,x4,x5,x6
0.1,0.002257,0.000675,0.000026,0.000019,0.000003,0.0
0.3,0.020491,0.011384,0.000373,0.000324,0.000048,0.0
0.5,0.038828,0.042895,0.001263,0.001220,0.000177,0.0


In [26]:
experiment_1.ivars_upp

,x1,x2,x3,x4,x5,x6
0.1,0.002261,0.000688,0.000031,0.000019,0.000003,0.0
0.3,0.022676,0.012021,0.000431,0.000338,0.000051,0.0
0.5,0.045247,0.047293,0.001420,0.001326,0.000200,0.0


**Reliability estimates of factor rankings based on VARS-TO**

reliability estimates give the ratio of how many times the rankings were the same for `bootstrap_size` sampling iterations

In [27]:
experiment_1.rel_sobol_factor_ranking

,x1,x2,x3,x4,x5,x6
0,1.0,1.0,0.82,0.82,1.0,1.0


In [28]:
experiment_1.rel_ivars_factor_ranking

,x1,x2,x3,x4,x5,x6
0.1,1.00,1.00,1.00,1.00,1.0,1.0
0.3,1.00,1.00,1.00,1.00,1.0,1.0
0.5,0.94,0.94,0.94,0.94,1.0,1.0


**Reliability estimates of factor rankings based on IVARS 50**

In [29]:
experiment_1.ivars50_grp

,x1,x2,x3,x4,x5,x6
0,2,2,1,2,2,1


In [30]:
experiment_1.sobol_grp

,x1,x2,x3,x4,x5,x6
0,2,1,2,2,1,2


**Reliability estimates of rankings based on grouping**

In [31]:
experiment_1.reli_sobol_grp

,x1,x2,x3,x4,x5,x6
0,1.0,1.0,1.0,1.0,1.0,1.0


In [32]:
experiment_1.reli_ivars50_grp

,x1,x2,x3,x4,x5,x6
0,1.0,1.0,0.94,0.94,1.0,1.0
